In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import sys
!{sys.executable} -m pip install optuna

     |████████████████████████████████| 365 kB 10.4 MB/s 
     |████████████████████████████████| 212 kB 26.2 MB/s 
     |████████████████████████████████| 78 kB 4.9 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7.4.3
    Uninstalling typing-extensions-3.7.4.3:
      Successfully uninstalled typing-extensions-3.7.4.3


In [3]:
import sys
!{sys.executable} -m pip install scikit-learn --upgrade

     |████████████████████████████████| 9.8 MB 10.9 MB/s 
     |████████████████████████████████| 297 kB 21.0 MB/s 
  Attempting uninstall: joblib
    Found existing installation: joblib 0.17.0
    Uninstalling joblib-0.17.0:
      Successfully uninstalled joblib-0.17.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.23.2
    Uninstalling scikit-learn-0.23.2:
      Successfully uninstalled scikit-learn-0.23.2


In [4]:
import sys
!{sys.executable} -m pip install scikit-learn-intelex

     |████████████████████████████████| 83 kB 1.0 MB/s 
     |████████████████████████████████| 12.6 MB 20.1 MB/s 
     |████████████████████████████████| 85.5 MB 48 kB/s 
     |████████████████████████████████| 4.0 MB 21.4 MB/s 


In [5]:
from sklearnex import patch_sklearn
patch_sklearn(global_patch=True)

Scikit-learn was successfully globally patched by Intel(R) Extension for Scikit-learn


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [6]:
import pandas as pd
import numpy as np

In [7]:
df = pd.read_csv('https://gitlab.com/mirsakhawathossain/exodata/-/raw/main/dataset/exodata.csv').sort_index(axis=0)

In [8]:
columns_1 = df.columns[df.mean()==1]
columns_2 = df.columns[df.mean()==0]
df = df.drop(columns=columns_1)
df = df.drop(columns=columns_2)

In [9]:
X = df.drop(['exoplanet'],axis=1)

In [10]:
X.shape

(75458, 741)

In [11]:
y = df[['exoplanet']]

In [12]:
y.value_counts()

exoplanet
1            40084
0            35374
dtype: int64

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=43,stratify=y)

In [15]:
print(X_train.shape)
print(X_test.shape)

(45274, 741)
(30184, 741)


In [1]:
#from sklearn.preprocessing import QuantileTransformer
#qt = QuantileTransformer()
from sklearn.preprocessing import StandardScaler
qt = StandardScaler()

In [17]:
X_train = qt.fit_transform(X_train)
X_test = qt.transform(X_test)

In [18]:
import optuna
from sklearn.model_selection import cross_val_score, RepeatedKFold
from sklearn.ensemble import RandomForestClassifier


In [19]:
def objective(trial):
    max_depth = trial.suggest_int('max_depth', 1, 5)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 6)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 5)
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2', None])
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
    n_estimators = trial.suggest_int('n_estimators', 50, 100, step=10)

    clf = RandomForestClassifier(
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        criterion=criterion,
        n_estimators=n_estimators,
        random_state=43,n_jobs = -1)

    rskf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=41)
    accs = np.mean(cross_val_score(clf, X_train, y_train, cv=rskf))
    return 1 - np.mean(accs)

In [20]:
# Define the study object to manage the optimization process
study = optuna.create_study(direction='minimize',pruner=optuna.pruners.HyperbandPruner(min_resource=1, max_resource='auto', reduction_factor=3))

[I 2023-03-28 02:07:47,358] A new study created in memory with name: no-name-951d3642-4712-4959-b599-f6a966cbaabe


In [21]:
# Run the hyperparameter optimization
n_trials = None
n_jobs = -1
timeout= 60
study.optimize(objective, n_trials=n_trials, n_jobs=n_jobs, timeout=timeout,show_progress_bar=True,gc_after_trial=True)

[I 2023-03-28 02:17:15,125] Trial 6 finished with value: 0.32898360838100427 and parameters: {'max_depth': 1, 'min_samples_split': 6, 'min_samples_leaf': 3, 'max_features': 'log2', 'criterion': 'gini', 'n_estimators': 50}. Best is trial 6 with value: 0.32898360838100427.
[I 2023-03-28 02:17:39,763] Trial 1 finished with value: 0.3255842775118113 and parameters: {'max_depth': 1, 'min_samples_split': 6, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'criterion': 'gini', 'n_estimators': 50}. Best is trial 1 with value: 0.3255842775118113.
[I 2023-03-28 02:17:41,895] Trial 2 finished with value: 0.31598284054797543 and parameters: {'max_depth': 2, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'log2', 'criterion': 'gini', 'n_estimators': 60}. Best is trial 2 with value: 0.31598284054797543.
[I 2023-03-28 02:17:47,576] Trial 7 finished with value: 0.3320537688038283 and parameters: {'max_depth': 1, 'min_samples_split': 6, 'min_samples_leaf': 3, 'max_features': 'log2', 'crite

In [22]:
# Print the best hyperparameters found
print('Best hyperparameters:')
print(study.best_params)

Best hyperparameters:
{'max_depth': 5, 'min_samples_split': 3, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'criterion': 'gini', 'n_estimators': 90}


In [23]:
best_score = 1 - study.best_value
print('Best score:', best_score)

Best score: 0.7131708871868488
